# Simple SD Webui Launcher

### One-click launch Stable Diffusion Webui for Google Colab. No accounts/tokens required. 

Select options then run the cell.

Cloning webui repository from https://github.com/AUTOMATIC1111/stable-diffusion-webui

Previous version of this notebook: https://colab.research.google.com/github/Miraculix200/StableDiffusionUI_Colab/blob/main/StableDiffusionUI_Colab_old.ipynb

## tl;dr infos



### Manual

Click button, wait a few minutes, click URL displayed in console output

### Models used in the settings

https://huggingface.co/Linaqruf/anything-v3.0  
https://huggingface.co/SpiteAnon/gigachad-diffusion  
https://huggingface.co/prompthero/midjourney-v4-diffusion  
https://huggingface.co/nitrosocke/mo-di-diffusion  
https://huggingface.co/TheLastBen/hrrzg-style-768px  
https://huggingface.co/acheong08/f222  
https://huggingface.co/acheong08/secretAI  
https://huggingface.co/hakurei/waifu-diffusion-v1-4  
https://huggingface.co/julienacquaviva/inpainting  
https://huggingface.co/acheong08/SD-V1-5-cloned  

### Description of options

`mount_gdrive`: Mounts Google Drive for permanent storage of some folder (embeddings, models, repositories)  
`install_default_settings`: Download some better default settings for the webui  
`install_image_browser_extension`: Installs image browser extension   
`install_tag_editor_extension`: Installs dataset tag editor extension useful for training embeddings/hypernetworks  
`use_localtunnel`: Use localtunnel reverse proxy instead of gradio  
`ngrok_token`: Enter a ngrok token here to use ngrok reverse proxy instead of localtunnel/gradio  


### Dreambooth support (beta)

Dreambooth support is messy on free Google Colab, because there is not enough system RAM to convert .ckpt to diffusers or diffusers to .ckpt. There is enough RAM and VRAM for some types of training however.

Use the `Import Model from Huggingface Hub` in the Dreambooth extension to clone a model from Huggingface to start training with.

After the training, when the webui crashes or becomes unresponsive, you have to run the cell at the end of this notebook which converts the diffusers into a Stable Diffusion checkpoint.

When the Dreambooth extension is installed, there may also be some errors while starting the webui. Dreambooth training will work anyway, so you can ignore them.

#### Recommended settings in the Dreambooth extension tab in the webui:

#### Create Model tab:

If you don't have Google Colab Pro, check `Import Model from Huggingface Hub` and enter `runwayml/stable-diffusion-v1-5` into the `Model Path` textbox

#### Train Model tab:

`Total number of classification images`: Number of (optional) class images you uploaded  
`Training steps`: 1200 for faces, 400 for objects (see [here](https://huggingface.co/blog/dreambooth#summary-of-initial-results))  
`Learning rate`: 0.000001 or 0.000002  
`Save a checkpoint every N steps`: 9000 (doesn't work properly right now)  
`Generate a preview image every N steps`: 9000  

#### Advanced:

These 2 options are mandatory, otherwise the webui will crash because of low VRAM

`Use 8bit Adam`: True  
`Mixed Precision`: fp16  

As of now (November 20th, 2022), you can leave the default values for most other options

See https://www.youtube.com/watch?v=HahKXY7AQ8c for a turorial how to use the extension

# Launch webui

Run this cell to start the webui

In [ ]:
#@markdown # Mount Google Drive to speed up webui launch
#@markdown Default folders on Google Drive: `embeddings`, `repositories`, `models`
mount_gdrive = True #@param {type:"boolean"}

#@markdown # Install default settings and extensions
#install_xformers = False #@param {type:"boolean"}
install_default_settings = True #@param {type:"boolean"}
install_image_browser_extension = True #@param {type:"boolean"}
install_tag_editor_extension = False #@param {type:"boolean"}
#install_dreambooth_extension = False #@param {type:"boolean"}
install_dreambooth_extension = False

#@markdown # Choose one or more models to download
#@markdown Set all to `False` if model is already present on Google Drive
model_15 = False #@param {type:"boolean"}
model_inpainting = False #@param {type:"boolean"}
model_anything_v3 = False #@param {type:"boolean"}
#model_anythingXtron = False #@param {type:"boolean"}
model_fred_herzog = False #@param {type:"boolean"}
model_f222 = False #@param {type:"boolean"}
model_secret_ai = False #@param {type:"boolean"}
model_waifu = False #@param {type:"boolean"}
model_gigachad = False #@param {type:"boolean"}
model_midjourney_v4 = False #@param {type:"boolean"}
model_modern_disney_style = False #@param {type:"boolean"}

#@markdown # Run localtunnel or ngrok reverse proxy instead of gradio
use_localtunnel = True #@param {type:"boolean"}
ngrok_token = "" #@param {type:"string"}
ngrok_region = "eu" #@param ["us", "eu", "ap", "au", "sa", "jp", "in"] {type:"raw"}

#@markdown # Additional commandline parameters and launch options
#@markdown See https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings
cmd_line_params = "--gradio-debug --disable-console-progressbars" #@param {type:"string"}
webui_port = 9001 #@param {type:"integer"}

#@markdown # Credentials for login to the webui (not necessary)
webui_username = "" #@param {type:"string"}
webui_password = "" #@param {type:"string"}

######################################################

CFG_URL = "https://raw.githubusercontent.com/Miraculix200/StableDiffusionUI_Colab/main/config.json"
CFG_UI_URL = "https://raw.githubusercontent.com/Miraculix200/StableDiffusionUI_Colab/main/ui-config.json"
XFORMERS_URL = "https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl"

# models which don't require a Huggingface token
MODEL_INPAINTING_URL = "https://huggingface.co/julienacquaviva/inpainting/resolve/main/sd-v1-5-inpainting.ckpt"
MODEL_15_URL = "https://huggingface.co/acheong08/SD-V1-5-cloned/resolve/main/v1-5-pruned-emaonly.ckpt"

webui_drive_folder = "/content/drive/MyDrive/SD_webui_Colab/"
webui_folder = "/content/stable-diffusion-webui/"

models_folder = webui_folder + "models/Stable-diffusion/"

######################################################

import os, time
from google.colab import drive

# list of folder to be linked to gdrive
def linkFoldersGdrive():
  mvToGdrive("embeddings")
  #mvToGdrive("extensions")
  #mvToGdrive("textual_inversion")
  mvToGdrive("repositories")
  mvToGdrive("models")
  #mvToGdrive("outputs")

# function for creating folders on gdrive and symlinks
def mvToGdrive(path_from_webui_root):
  if not os.path.exists(webui_drive_folder + path_from_webui_root):
    print("Creating folder: " + webui_drive_folder + path_from_webui_root)
    !mkdir -p {webui_drive_folder}{path_from_webui_root}

  if not os.path.islink(webui_folder + path_from_webui_root):
    if os.path.exists(webui_folder + path_from_webui_root):
      print("Copying folder: " + webui_folder + path_from_webui_root)
      !cp -r {webui_folder}{path_from_webui_root} {webui_drive_folder}
      print("Deleting folder: " + webui_folder + path_from_webui_root)
      !rm -rf {webui_folder}{path_from_webui_root}

    !ln -sv {webui_drive_folder}{path_from_webui_root} {webui_folder}{path_from_webui_root}

######################################################

# check GPU availability
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

# # install xformers
# if install_xformers:
#   %pip install -q {XFORMERS_URL}
#   %pip install -q -U --pre triton
#   cmd_line_params = cmd_line_params + " --force-enable-xformers"

# mount google drive
if mount_gdrive:
  drive.mount('/content/drive')

# clone webui repo
!git clone --depth 1 https://github.com/AUTOMATIC1111/stable-diffusion-webui {webui_folder}

# download default settings
if install_default_settings:
  %cd /content/stable-diffusion-webui
  if not os.path.exists(webui_folder + "config.json"):
    !wget {CFG_URL}
  if not os.path.exists(webui_folder + "ui-config.json"):
    !wget {CFG_UI_URL}

# install image browser extension
if install_image_browser_extension:
  %cd {webui_folder}extensions
  if not os.path.exists('stable-diffusion-webui-images-browser'):
    !git clone --depth 1 https://github.com/yfszzx/stable-diffusion-webui-images-browser

# install Dataset Tag Editor extension
if install_tag_editor_extension:
  %cd {webui_folder}extensions
  if not os.path.exists('stable-diffusion-webui-dataset-tag-editor'):
    !git clone --depth 1 https://github.com/toshiaki1729/stable-diffusion-webui-dataset-tag-editor.git

# dreambooth
if install_dreambooth_extension:
  !mkdir -p /content/dreambooth_ext/dataset_images
  !mkdir -p /content/dreambooth_ext/class_images
  
  %cd /content/stable-diffusion-webui/extensions
  !git clone --depth 1 https://github.com/d8ahazard/sd_dreambooth_extension.git

  %pip install -qq git+https://github.com/ShivamShrirao/diffusers

  %pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
  %pip install -q -U --pre triton
  %pip install -q accelerate==0.12.0 bitsandbytes

  %pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
  # These were compiled on Tesla T4, should also work on P100, thanks to https://github.com/metrolobo

# copy/link folders to gdrive
if mount_gdrive:
  linkFoldersGdrive()

#  # temporarily copy repositories from gdrive to session filesystem
#  # note sure if necessary, so they can be updated by git
#  repo_path = webui_folder + "repositories"
#  if os.path.islink(repo_path):
#    print("Deleting repositories symlink and copying repositories/ folder to local fs")
#    !rm {webui_folder}repositories
#    !cp -rp {webui_drive_folder}repositories {webui_folder}repositories
#    print("Installing requirements")
#    # install requirements and exit
#    %cd {webui_folder}
#    !COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python launch.py
#    mvToGdrive("repositories")

######################################################

# download models
%cd {models_folder}

# download models
if model_15:
  !wget {MODEL_15_URL}
  !wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O v1-5-pruned-emaonly.vae.pt 

if model_inpainting:
  !wget {MODEL_INPAINTING_URL}

if model_anything_v3:
  !wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0-pruned.ckpt
  !wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0.vae.pt -O Anything-V3.0-pruned.vae.pt

#if model_anythingXtron:
#  !wget https://huggingface.co/CrasHthe2nd2/anything-tron/resolve/main/anythingXtron.ckpt
#  !wget https://huggingface.co/CrasHthe2nd2/anything-tron/resolve/main/Anything-V3.0.vae.pt -O anythingXtron.vae.pt

if model_gigachad:
  !wget https://huggingface.co/SpiteAnon/gigachad-diffusion/resolve/main/gigachad_2000.ckpt

if model_midjourney_v4:
  !wget https://huggingface.co/prompthero/midjourney-v4-diffusion/resolve/main/mdjrny-v4.ckpt

if model_modern_disney_style:
  !wget https://huggingface.co/nitrosocke/mo-di-diffusion/resolve/main/moDi-v1-pruned.ckpt

if model_fred_herzog:
  !wget https://huggingface.co/TheLastBen/hrrzg-style-768px/resolve/main/hrrzg.ckpt

if model_f222:
  !wget https://huggingface.co/acheong08/f222/resolve/main/f222.ckpt

if model_secret_ai:
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/animefull-final-pruned/model.ckpt -O secret_ai.ckpt
    %cd {webui_folder}models/VAE/
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/animevae.pt
    !mkdir -p {webui_folder}/models/hypernetworks/
    %cd {webui_folder}models/hypernetworks/
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/aini.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/anime.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/anime_2.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/anime_3.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_2.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_3.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_kemono.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_protogen.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_scalie.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_transformation.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/pony.pt

if model_waifu:
  %cd {models_folder}
  !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/models/wd-1-3-penultimate-ucg-cont.ckpt
  %cd {webui_folder}models/VAE/
  !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt
  !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt

# count number of .ckpt files in models folder
ckpt_count = 0

for root, dirs, files in os.walk(models_folder):
    for file in files:
        if file.endswith(".ckpt"):
             ckpt_count += 1

# download model when no models are detected
if ckpt_count < 1:
  print('[1;31mNo models detected. Downloading default model')
  %cd {models_folder}
  !wget https://huggingface.co/SpiteAnon/gigachad-diffusion/resolve/main/gigachad_2000.ckpt

######################################################

# install and launch localtunnel
if ngrok_token != "":
  cmd_line_params = cmd_line_params + " --ngrok " + ngrok_token + " --ngrok-region " + ngrok_region
elif use_localtunnel:
  if not os.path.exists('/tools/node/bin/lt'):
    print("Installing localtunnel")
    !npm install -g localtunnel
  !nohup lt --port {str(webui_port)} > /content/srv.txt 2>&1 &
  time.sleep(2)
  print("The webui will run on this URL:")
  !grep -o 'https[^ ]*' /content/srv.txt
else:
  cmd_line_params = cmd_line_params + " --share"

######################################################

# launch webui
%cd {webui_folder}

cmd_line_params = cmd_line_params + " --port " + str(webui_port)

if webui_username != "":
  cmd_line_params = cmd_line_params + " --gradio-auth " + webui_username + ":" + webui_password

!sed -i -e 's/numpy==1.23.3/numpy==1.21.6/g' requirements_versions.txt
!sed -i -e 's/fairscale==0.4.9/fairscale==0.4.6/g' requirements_versions.txt

!python launch.py {cmd_line_params}

######################################################

# Optional cells

## Zip images for downloading on local drive
After running the cell, click the folder icon on the left, the one below {x}, search for `outputs.zip` in the `/content` folder

In [ ]:
zip_folder = "/content/stable-diffusion-webui/outputs" #@param {type:"string"}
!zip -r /content/outputs.zip {zip_folder}

## Delete Google Drive permanent storage

This cell removes `SD_webui_Colab` folder from Google Drive

In [ ]:
drive_folder = "SD_webui_Colab"  #@param {type:"string"}
if drive_folder == "":
    print('[1;31mgood morning')
else:
    !rm -rf /content/drive/MyDrive/{drive_folder}
    print("Folder deleted: " + drive_folder)

## Checkout pull request from AUTOMATIC1111 stable-diffusion-webui repository

In [ ]:
pullrequest_number = "4509" #@param {type:"string"}

import os

if not os.path.exists("/content/stable-diffusion-webui/"):
    %cd /content/
    !git clone --depth 1 https://github.com/AUTOMATIC1111/stable-diffusion-webui stable-diffusion-webui

%cd /content/stable-diffusion-webui/
!git pull origin pull/{pullrequest_number}/head

## Stable Diffusion Pickle Scanner: scan models for malware

Scan .pt, .ckpt and .bin files for potentially malicious code

Source: https://github.com/zxix/stable-diffusion-pickle-scanner



In [ ]:
scan_folder = "/content/stable-diffusion-webui/models" #@param {type:"string"}
%cd /content/stable-diffusion-webui
!wget https://raw.githubusercontent.com/zxix/stable-diffusion-pickle-scanner/main/pickle_scan.py
!wget https://raw.githubusercontent.com/zxix/stable-diffusion-pickle-scanner/main/pickle_inspector.py

!python pickle_scan.py {scan_folder}

## Download a model from Huggingface with token

If the above downloads without token didn't work, these models can be downloaded after accepting their license on Huggingface and entering the token here

In [ ]:
huggingface_token = "" #@param {type:"string"}
user_header = f"\"Authorization: Bearer {huggingface_token}\""

!mkdir -p /content/stable-diffusion-webui/models/Stable-diffusion
%cd /content/stable-diffusion-webui/models/Stable-diffusion

# model 1.5
!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt 
!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O v1-5-pruned-emaonly.vae.pt 

# inpaint model 1.5
!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt

## Download a model from Google Drive

Copy the file ID from your url and paste it into the GOOGLE_FILE_ID text field below, before running this cell

In [ ]:
###########################################################
google_file_id = "1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA" #@param {type:"string"}
download_folder = "/content/stable-diffusion-webui/models/Stable-diffusion/" #@param {type:"string"}
###########################################################

# above file ID is Stable Diffusion model 1.4. Replace it with the file ID of the model you want to download from Google Drive

%cd {download_folder}
url = "https://drive.google.com/u/0/uc?id=" + google_file_id + "&export=download&confirm=t"
!gdown $url

## Download a model from MEGA


Change MEGA_DOWNLOAD_URL to the URL of the model, then run the cell

Download can be very slow (>30 minutes for a 4GB model)

In [ ]:
##########################################################
mega_download_url = "https://mega.nz/file/mdcVARhL#FUq5TL2xp7FuzzgMS4B20sOYYnPZsyPMw93sPMHeQ78 " #@param {type:"string"}
download_folder = "/content/stable-diffusion-webui/models/Stable-diffusion/" #@param {type:"string"}
##########################################################

%cd {download_folder}

%pip install mega.py
from mega import Mega

mega = Mega()
print("Connecting to MEGA")
m = mega.login()
print("Downloading file (no progress display available)")
m.download_url(mega_download_url)
print("Download complete")

## Dreambooth utility: Convert .ckpt to diffusers

In [ ]:
#@markdown Enter .ckpt path of the model you want to convert
sd_ckpt_path = "/content/stable-diffusion-webui/models/Stable-diffusion/gigachad_2000.ckpt" #@param {type:"string"}
#@markdown Enter path to the dreambooth models folder
sd_dreambooth_path = "/content/stable-diffusion-webui/models/dreambooth/" #@param {type:"string"}
#@markdown The name of the created model (select this in the webui Dreambooth tab)
my_model_name = "MyNewModel" #@param {type:"string"}
#@markdown Scheduler (don't change unless you know what you're doing)
scheduler = "ddim" #@param {type:"string"}

import os

if os.path.exists(sd_ckpt_path):

  dump_path = sd_dreambooth_path + my_model_name + "/working"
  !mkdir -p {dump_path}

  # install necessary packages
  %pip install -qq omegaconf pytorch-lightning transformers ftfy

  # get conversion script and start conversion
  %cd /content/dreambooth_ext/
  !wget https://raw.githubusercontent.com/ShivamShrirao/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
  !python convert_original_stable_diffusion_to_diffusers.py --checkpoint_path {sd_ckpt_path} --dump_path {dump_path}

  # get v1-inference.yaml
  %cd {dump_path}
  !wget "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml" -O v1-inference.yaml

  # create db_config.json
  json_content = '{"model_name": "' + my_model_name + '", "scheduler": "' + scheduler + '", "src": "hax", "total_steps": 0}'
  #json_content = '{"model_name": "' + my_model_name + '", "scheduler": "ddim", "src": "v1-5-pruned-emaonly.ckpt [81761151]", "total_steps": 0}'

  %cd ..
  with open('db_config.json', 'w') as f:
      f.write(json_content)

else:
  print('[1;31mFile not found:')
  print(sd_ckpt_path)

## Dreambooth utility: Convert diffusers to .ckpt

In [ ]:
#@markdown ### (beta) Convert diffusers model to Stable Diffusion checkpoint
#@markdown If you did a Dreambooth training on free Google Colab, the webui will crash when the training is complete, because there is not enough system RAM to convert the diffusers to a Stable Diffusion checkpoint

#@markdown Use Google Colab Pro to prevent this

#@markdown Run this cell to start the conversion

#@markdown #### Select this option to copy the .ckpt file to your Google Drive

copy_ckpt_to_gdrive = True #@param {type:"boolean"}

#@markdown #### Enter the name of the model you created in Dreambooth extension

my_model_name = "MyNewModel" #@param {type:"string"}

if copy_ckpt_to_gdrive:
    print("Give permission to access Google Drive")
    from google.colab import drive
    drive.mount('/content/drive')

%cd /content/dreambooth_ext/
!wget https://raw.githubusercontent.com/ShivamShrirao/diffusers/main/scripts/convert_diffusers_to_original_stable_diffusion.py

model_p = "/content/stable-diffusion-webui/models/dreambooth/" + my_model_name + "/working"
ckpt_p = "/content/stable-diffusion-webui/models/Stable-diffusion/" + my_model_name + ".ckpt"

print("Converting diffusers to original Stable Diffusion checkpoint")

!python convert_diffusers_to_original_stable_diffusion.py --model_path {model_p} --checkpoint_path {ckpt_p}

print("Saved checkpoint to " + ckpt_p)

if copy_ckpt_to_gdrive:
  print("Copying " + my_model_name + ".ckpt to Google Drive")
  !cp {ckpt_p} /content/drive/MyDrive/
  print("Done")